In [2]:
%cd /home/rwb/github/sdp-par-model
import sys
# sys.path.extend(['/home/rwb/github/sdp-par-model', '/home/rwb/github/skaworkflows'])
from sdp_par_model import reports
from sdp_par_model.parameters.definitions import *


/home/rwb/github/sdp-par-model


In [10]:
output_dir = "/home/rwb/github/skaworkflows/skaworkflows/data/sdp-par-model_output/"
reports.write_csv_hpsos(f'{output_dir}/2023-03-19_long_HPSOs.csv', HPSOs.available_hpsos, parallel=0, verbose=False)
# reports.write_csv_hpsos(f'{output_dir}/2023-03-19_mid_HPSOs.csv', HPSOs.available_hpsos, parallel=0, verbose=True, adjusts={'Bmax': 32500})
# reports.write_csv_hpsos(f'2023-03-19_x_HPSOs.csv', HPSOs.available_hpsos, parallel=0, verbose=True, adjusts={'Bmax': 16250})
# reports.write_csv_hpsos(f'{output_dir}/2023-03-19_short_HPSOs', HPSOs.available_hpsos, parallel=0, verbose=True, adjusts={'Bmax': 4062.5})

DFT
N 14400
T 600
Rflop 4.51214341732121e+15
Rflop_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79d4d430>
Rout 3310508671096.60
Rout_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79a86a00>
Rvis 29290.0505917751
Rvis_bdep 3831607258.21366
Rvis_bdep_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79a6f520>
Rvis_N 55175144518276.7
Rvis_N_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79a80b80>
N:14400
ICAL
Solve
N 40
T 600
Rflop 157026519376431.
Rout 5461333.33333333
Rvis 29290.0505917751
Rvis_bdep 10643353.4950379
Rvis_bdep_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79ae8640>
Rvis_N 153264290328.546
Rvis_N_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79a80b80>
DFT
N 17600
T 600
Rflop 2.04262535642661e+15
Rflop_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79ba7d90>
Rout 749325578647.792
Rout_task <sdp_par_model.parameters.container.BLDep object at 0x7fcd79a91910>
Rvis 

In [9]:
Rvis = 29290.0505917751
Rvis_bdep = 3831607258.21366
Rvis_N = 6130571613141.85

Rvis_N / 1600
Rvis_bdep / (10**6)
# Rvis / (10**6)

3831.60725821366